## TTC Subway Delay Analysis Project

In [2]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
# ttc_codes = pd.read_csv('CSV Files\Code Descriptions.csv')
# ttc_codes.head()

ttc_codes = pd.read_json('CSV Files\Code Descriptions.json')
ttc_codes.head(10)

ttc_subway_delay_24 = pd.read_csv('CSV Files/ttc-subway-delay-data-2024.csv')
ttc_subway_delay_24.head(10)

# Updated as of Nov 24th, 2025
ttc_subway_delay_25 = pd.read_csv('CSV Files/TTC Subway Delay Data since 2025.csv')
ttc_subway_delay_25.head(10)

,_id,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle
0,1,2025-01-01,02:10,Wednesday,BATHURST STATION,MUSAN,5,9,E,BD,5227
1,2,2025-01-01,02:30,Wednesday,DUNDAS STATION,MUIRS,0,0,NaN,YU,0
2,3,2025-01-01,02:32,Wednesday,BROADVIEW STATION,PUMST,0,0,E,BD,0
3,4,2025-01-01,02:58,Wednesday,KEELE STATION,EUSC,0,0,W,BD,5293
4,5,2025-01-01,02:58,Wednesday,COXWELL STATION,SUAE,0,0,NaN,BD,0
5,6,2025-01-01,07:59,Wednesday,DONLANDS STATION,TUNOA,5,10,W,BD,5000
6,7,2025-01-01,08:13,Wednesday,BLOOR STATION,PUTR,9,0,S,YU,5836
7,8,2025-01-01,08:15,Wednesday,BLOOR STATION,PUTR,5,0,N,YU,5706
8,9,2025-01-01,09:14,Wednesday,FINCH STATION,SUDP,0,0,S,YU,5596
9,10,2025-01-01,09:22,Wednesday,BATHURST STATION,SUUT,0,0,E,BD,5312
